In [1]:
import numpy as np
import pandas as pd
import networkx as nx

from tqdm import tqdm

# Load the data

In [2]:
ratings_df = pd.read_csv('../data/Ratings.csv', delimiter=';')

In [3]:
books_df = pd.read_csv('../data/Books.csv', delimiter=';')

# Preprocess data

In [4]:
# filter users and books with less than 5 interactions
user_threshold = 5
book_threshold = 5

user_counts = ratings_df['User-ID'].value_counts()
book_counts = ratings_df['ISBN'].value_counts()

filtered_users = user_counts[user_counts >= user_threshold].index
filtered_books = book_counts[book_counts >= book_threshold].index

user_mask = ratings_df['User-ID'].isin(filtered_users)
book_mask = ratings_df['ISBN'].isin(filtered_books)

ratings_df = ratings_df[user_mask & book_mask]

# Init the bipartite graph

Let $G (V, E)$ be a bipartite graph with vertices $V = V_I \sqcup V_U$, where 

- $V_I$ is the set of nodes corresponding to books, 
- $V_U$ is the set of nodes corresponding to users,

and edge set $E = \left\{ ui \ | \ R(u, i); u \in V_U, i \in V_I\right\}$, where $R(u, i)$ is 1 if user $u$ has rated book $i$, and 0 otherwise.

In [5]:
# init a graph
G = nx.Graph()

# form a bipartite graph with users and books being the two partitions
G.add_nodes_from(filtered_users, bipartite=0)
G.add_nodes_from(filtered_books, bipartite=1)

# add edges between users and books based on interactions
edges = list(ratings_df[['User-ID', 'ISBN']].itertuples(index=False, name=None))
G.add_edges_from(edges)

# PageRank Algorithm

The Page Rank score for a vertex $v_i$ is iteratively calculated as follows:
$$
r(v_i) = \frac{1 - \alpha}{|V|} + \alpha \sum_{v_j \in N^-(v_i)} \frac{r(v_j)}{d^+(v_j)}
$$
where:
- $|V|$ is the number fo vertices in a graph
- $N^-(v)$ is the in-neighborhood of a vertex $v$ (a set of vertices that have outgoing edge to $v$)
- $d^+(v)$ is the out-degree of a vertex $v$ (number of vertices that can be reached from $v$ in one step)
- $\alpha$ is a weight of pure PageRank score ("damping factor")

In [6]:
def pagerank(G, alpha=0.5, max_iter=100, tol=1e-06):

    nodes = G.nodes()
    N = len(nodes)

    # init the PageRank dict with equal ranks
    pagerank = {node: 1 / N for node in nodes}

    for i in tqdm(range(max_iter)):
        prev_pagerank = pagerank.copy()
        for node in nodes:
            rank_sum = 0
            for neighbor in G.neighbors(node):
                n_neighbors = len(list(G.neighbors(neighbor)))
                rank_sum += prev_pagerank[neighbor] / n_neighbors
            pagerank[node] = (1 - alpha) / N + alpha * rank_sum

        # get absolute differences between ranks and check for convergence
        rank_diffs = np.array([
            np.abs(pagerank[node] - prev_pagerank[node]) for node in nodes
        ])
        if rank_diffs.sum() < tol:
            break

    return pagerank


# Calculate PageRank scores

In [7]:
# Calculate PageRank
pr = pagerank(G)

 19%|█▉        | 19/100 [01:13<05:12,  3.86s/it]


In [8]:
pr_df = pd.DataFrame(list(pr.items()), columns=['node', 'pagerank'])

In [9]:
user_pr_mask = pr_df['node'].isin(filtered_users)

user_pr = pr_df[user_pr_mask].sort_values('pagerank', ascending=False)
book_pr = pr_df[~user_pr_mask].sort_values('pagerank', ascending=False)

In [10]:
top_books = book_pr.head(10)

top_books_df = pd.merge(top_books, books_df, left_on='node', right_on='ISBN')
top_books_df

,node,pagerank,ISBN,Title,Author,Year,Publisher
0,0971880107,0.000999,0971880107,Wild Animus,Rich Shapero,2004,Too Far
1,0316666343,0.000380,0316666343,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown"
2,0385504209,0.000282,0385504209,The Da Vinci Code,Dan Brown,2003,Doubleday
3,0060928336,0.000222,0060928336,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Rebecca Wells,1997,Perennial
4,0312195516,0.000207,0312195516,The Red Tent (Bestselling Backlist),Anita Diamant,1998,Picador USA
5,059035342X,0.000200,059035342X,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books
6,0671027360,0.000186,0671027360,Angels & Demons,Dan Brown,2001,Pocket Star
7,0142001740,0.000186,0142001740,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books
8,044023722X,0.000185,044023722X,A Painted House,John Grisham,2001,Dell Publishing Company


# Results analysis

Books with high PageRank scores are likely to be popular, meaning they are rated by users who have also interacted with other important books.

Users with high PageRank scores are likely to read important books, meaning they are highly influential on the books score.

PageRank algorithm can be used to solve cold start problem, identify popular and influential books and users to customize recommendations; the underlying graph structure can help identify users and book clusters.